# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов телеком компании, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Требуется построить модель с максимально большим значением метрики *accuracy*. Чтобы считать проект успешным, нужно довести долю правильных ответов по крайней мере до 0.75.

Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
 - сalls — количество звонков,
 - minutes — суммарная длительность звонков в минутах,
 - messages — количество sms-сообщений,
 - mb_used — израсходованный интернет-трафик в Мб,
 - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 

## Откройте и изучите файл

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv') # читаем csv-файл и сохраняем в переменную df
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv') 

df.sample(5)  # смотрим рандомные 5 строк данных

,calls,minutes,messages,mb_used,is_ultra
403,40.0,285.77,6.0,19721.43,0
2181,76.0,524.56,0.0,5291.10,1
2575,74.0,617.02,69.0,20878.34,0
754,98.0,723.12,10.0,14688.44,0
2292,67.0,560.62,68.0,16663.57,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:** Форматы данных адекватные. Пропусков нет, можно использовать данные для обучения.

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод:** Отрицательных значений в столбцах со звонками, минутами, сообщениями и мб нет, данные выглядят адекватными.

## Разбейте данные на выборки

Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [5]:
features = df.drop(['is_ultra'], axis=1) 
target = df['is_ultra']
# у нас ситуация, когда есть исходные данные и нужно их поделить на 3 выборки: делим в соотношении 3:1:1 (60%, 20%, 20%). 
# Метод train_test_split не дает возможности сразу разделить данные на 3 части, поэтому применяем его 2 раза последовательно:  
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345) # отделяем 20% данных для тестовой выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345) # отделяем 20% (25% от оставшихся) 
                                                                                  # данных для валидационной выборки

## Исследуем модели

Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

In [6]:
# дерево решений

best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
    result = round(model.score(features_valid, target_valid), 2) # считаем качество модели на валидационной выборке
    if result > best_tree_result:
        best_tree_model = model # сохраняем наилучшую модель
        best_tree_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных
        best_tree_depth = depth #  сохраняем наилучшее значение гиперпараметра depth на валидационных данных
print("Качество лучшего дерева решений на валидационной выборке:", best_tree_result, "Глубина дерева:", best_tree_depth)


Качество лучшего дерева решений на валидационной выборке: 0.77 Глубина дерева: 3


In [7]:
# случайный лес

best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
        result = round(model.score(features_valid, target_valid), 2) # считаем качество модели на валидационной выборке
        if result > best_forest_result:
            best_forest_model = model # сохраняем наилучшую модель
            best_forest_result = result #  сохраняем наилучшее значение метрики accuracy на валидационных данных
            best_forest_est = est #  сохраняем наилучшее значение гиперпараметра n_estimators на валидационных данных
            best_forest_depth = depth #  сохраняем наилучшее значение гиперпараметра depth на валидационных данных
print("Качество лучшего случайного леса на валидационной выборке:", best_forest_result, "Количество деревьев:", best_forest_est,\
"Максимальная глубина:", best_forest_depth)


Качество лучшего случайного леса на валидационной выборке: 0.8 Количество деревьев: 20 Максимальная глубина: 10


In [8]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100) 
model.fit(features_train, target_train) 
result = round(model.score(features_valid, target_valid), 2)

print("Качество логистической регрессии на валидационной выборке:", result)

Качество логистической регрессии на валидационной выборке: 0.73


**Вывод:** Худший показатель качества модели у логистической регрессии (0.73), однако, скорость вычислений была наиболее высокой. Второй результат по качеству (0.77) у дерева решений при глубине дерева 3, скорость вычислений так же средняя. Первый результат по качеству у случайного леса (0.8) при количестве деревьев 20 и максимальной глубине 10, однако и скорость вычислений самая низкая.

## Проверка модели на тестовой выборке

In [9]:
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=10)
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
test_result = round(model.score(features_test, target_test), 2) # проверяем модель на тестовой выборке
print("Качество модели на тестовой выборке:", test_result)

Качество модели на тестовой выборке: 0.79


**Вывод:** Качество модели немного упало на тестовой выборке по сравнению с результатами на валидационной выборке. Тем не менее значение параметра не меньше 0.75. Цель достигнута.

## Проверка модели на адекватность

Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

In [10]:
#внедрим baseline модель на основе DummyClassifier из пакета sklearn со стратегией most_frequent
model = DummyClassifier(strategy='most_frequent', random_state=12345)
model.fit(features_train, target_train)
result_dumclass = round(model.score(features_test, target_test), 2)
print("Качество модели DummyClassifier:", result_dumclass)

Качество модели DummyClassifier: 0.7


**Комментарий:** Качество более наивной модели на тестовой выборке 0.7, а выбранной модели 0.79, что подтверждает ее адекватность и более качественную работу.